In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# read data from a file
df = pd.read_csv('diabetes_2.csv')

In [2]:
df.head()

,Unnamed: 0,Age,Gender,BMI,High_BP,FBS,HbA1c_level,Smoking,Diagnosis
0,0,80,Female,25,0,140,6.6,0,0
1,1,54,Female,27,0,80,6.6,0,0
2,2,28,Male,27,0,158,5.7,0,0
3,3,36,Female,23,0,155,5.0,1,0
4,4,76,Male,20,1,155,4.8,1,0


In [10]:
# split the data into X and y
X = df.drop('Diagnosis', axis=1)
y = df['Diagnosis']

In [11]:
# convert categorical variables to numerical using one-hot encoding
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
# for loop object and catagory in the data
for col in X.columns:
    if X[col].dtype == 'object' or X[col].dtype.name == 'category':
        X[col] = labelencoder.fit_transform(X[col])

In [12]:
X

,Unnamed: 0,Age,Gender,BMI,High_BP,FBS,HbA1c_level,Smoking
0,0,80,0,25,0,140,6.6,0
1,1,54,0,27,0,80,6.6,0
2,2,28,1,27,0,158,5.7,0
3,3,36,0,23,0,155,5.0,1
4,4,76,1,20,1,155,4.8,1
...,...,...,...,...,...,...,...,...
88375,88375,17,0,15,0,100,5.7,1
88376,88376,22,1,19,0,120,6.4,1
88377,88377,27,0,24,1,140,7.1,1
88378,88378,32,1,29,1,160,7.8,1


In [13]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Scaling data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
# Base models
estimators = [
    ('logistic', LogisticRegression(max_iter=1000, solver='lbfgs')),
    ('svm', SVC(kernel='rbf'))
]

# Meta-model (stacking classifier)
stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

# fit the stacking model
stacking_model.fit(X_train, y_train)

# make predictions
stacking_pred = stacking_model.predict(X_test)

# Calculate accuracy for stacking model
stacking_accuracy = accuracy_score(y_test, stacking_pred)

print("Stacking Testing Accuracy:", stacking_accuracy)

c:\Users\usama\miniconda3\envs\ML_WORK\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\usama\miniconda3\envs\ML_WORK\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

Stacking Testing Accuracy: 0.9532133966960851


In [ ]:
# code to save model as pickle file
import pickle
pickle.dump(stacking_model, open('clf_model.pkl', 'wb'))